In [1]:
import gurobipy as gp
from gurobipy import GRB, quicksum

In [2]:
# pip install gurobipy

# 저자

- TA: 성준모 (Joonmo Sung)
- SYSTEMS MODELING AND PROGRAMMING LAB DEPARTMENT OF INDUSTRIAL ENGINEERING, YONSEI UNIVERSITY - SYMPLY
- 문의: `sjm21314@naver.com`,`sjm21314@yonsei.ac.kr`
***
본 강의자료는 Keha,A. B.,Khowala,K.,&Fowler,J. W. (2009). Mixed integer programming formulations for single machine scheduling problems. Computers & Industrial Engineering, 56(1), 357-367 논문을 바탕으로 제작되었으며,

반도체데이터사이언스 협동과정 수업 용도 이외의 목적으로 저자의 허락 없이 다른 사람들과 공유할 수 없습니다.

## Dataset
### Set
- N = $\{1,2,...,n\}, j \in N$
### Parameters
- $p_j$: processing time of job $j$
- $d_j$: due date of job $j$
- $w_j$: weight of job $j$
- $r_j$: release date of job $j$

In [3]:
# Offline 상황을 가정.

n = 5 # 작업의 개수

N = [_ for _ in range(n)] # 작업 set을 리스트 자료구조를 이용해 0, 1, 2,..., n-1 까지 담아줌
p = [4, 3, 7, 2, 2] # 가공시간 데이터를 리스트 자료구조를 이용해 만들어 줌
d = [5, 6, 8, 8, 17] # 납기 데이터를 리스트 자료구조를 이용해 만들어 줌
w = [5, 4, 3, 2, 1] # 가중치 데이터를 리스트 자료구조를 이용해 만들어 줌
r = [1, 2, 3, 3, 5] # 작업의 release date(도착 시점) 데이터를 리스트 자료구조를 이용해 만들어 줌

M = (sum(p) + sum(r)) * 2 # big M

In [4]:
import numpy as np

# random seed를 정해줌. 고정시 계속 같은 data가 만들어짐
np.random.seed(917)
n = 10
# L, R은 임의로 고름
L = 0.5 # {0.5, 0.7} 중 하나를 고르면 됨. release 데이터에서는 0.5로 고정
R = 0.8 # {0.4, 0.8, 1.4} 중 하나를 고르면 됨
Q = 0.4 # Q는 0.4로 고정


N = [_ for _ in range(n)] # 작업 set을 리스트 자료구조를 이용해 0, 1, 2,..., n-1 까지 담아줌
# Random generation 코드
p = np.random.randint(1, 101, size=n) # processing time은 [1, 100] uniform 분포 따름
w = np.random.randint(1, 11, size=n) # weight은 [1, 10] uniform 분포 따름
d = np.random.randint(int(sum(p)*(L-R/2)), int(sum(p)*(L+R/2)) + 1, size = n) # [P(L - R/2), P(L + R/2)] 분포를 따름
r = np.random.randint(0, int(Q * sum(p)) + 1, size=n) # [0, QP] 분포를 따름

M = (sum(p) + sum(r)) * 2 # big M

print(p)
print(w)
print(d)
print(r)

[88 91 65 22 63 13  9 70 12 16 18 62 82 55 30]
[ 1  5  2  5  3  5  5  5 10  2  5  8  9 10  6]
[576 186 527 615  79 527 355 281  77 497 238 103 391  95 276]
[186  22 147 214 184 176 163 246 263 205 142 265 176  72 122]


In [5]:
# Create an environment with your WLS license
# params = {
# "WLSACCESSID": 'your wls accessid (string)',
# "WLSSECRET": 'your wls secret (string)',
# "LICENSEID": <your license id (integer)>,
# }
# env = gp.Env(params=params)

# # Create the model within the Gurobi environment
# model = gp.Model(env=env)

In [6]:
obj_select = 'twt' # 'wct', 'tj', 'twt', 'L_max' 중 하나 선택
release_time = False

## 1. Completion time variables

### Decision Variables
- $C_j$: 작업 $j$의 시작 시점 결정 $C_j \geq 0 \ \forall j \in N$
- $y_{j,k}$: 작업 $j$가 $k$ 전에 작업 중인지 후에 작업중인지를 결정 $y_{j,k} \in \{0, 1\} \ \forall j,k \in N \ and \ j < k $

In [7]:
# 모델 선언
model_MIP_1 = gp.Model("MIP_1") # 먼저 모델 선언 

# colab 경우
# model_MIP_1 = gp.Model("MIP_1", env = env) 

# Decision Variable 만들기
C = {(j): model_MIP_1.addVar(vtype=GRB.CONTINUOUS, lb = 0, name=f"C_{j+1}") for j in N} # 연속 변수, lower bound 0 (0 이상)
y =  {(j, k): model_MIP_1.addVar(vtype=GRB.BINARY, name=f"y_{j+1}_{k+1}") for j in N for k in N if j < k} # 이진 변수, j < k 는 그대로 써주면 됨

Set parameter Username
Set parameter LicenseID to value 2604879
Academic license - for non-commercial use only - expires 2026-01-03


### 기본 제약식
- $C_j \geq p_j \quad \forall j \in N$
- $C_j + p_k \leq C_k + M(1-y_{j,k}) \quad \forall j,k \in N, j < k$
- $C_k + p_j \geq C_j + My_{j,k} \quad \forall j,k \in N, j < k$

In [8]:
# 제약식
constraint1 =  {(j): model_MIP_1.addConstr(C[j] >= p[j], name = f"constraint1_{j}") for j in N}
constraint2 =  {(j, k): model_MIP_1.addConstr(C[j] + p[k] <= C[k] + M*(1-y[j,k]), name = f"constraint2_{j}_{k}") for j in N for k in N if j < k}
constraint3 =  {(j, k): model_MIP_1.addConstr(C[k] + p[j] <= C[j] + M*y[j,k], name = f"constraint3_{j}_{k}") for j in N for k in N if j < k}

#### Release time 반영 제약식
$ C_j \geq p_j + r_j \quad \forall j \in N$

In [9]:
# 문제에 release time을 반영하려면 아래 제약식 추가
if release_time == True:
    constraint4 = {(j): model_MIP_1.addConstr(C[j] >= p[j] + r[j], name = f"constraint_{j}") for j in N}

### 목적식

#### To minimize the total weighted completion time
$ Minimize \sum_{j = 1}^n w_j C_j$

In [10]:
if obj_select == "wct":
    # 목적식
    model_MIP_1.setObjective(quicksum(w[j]*C[j] for j in N), GRB.MINIMIZE)

#### To minimize $L_{max}$
$ Minimize \ L_{max}$
$ \\ s.t. \  L_{max} \geq (C_j - d_j) \quad \forall j \in N$

In [11]:
if obj_select == "L_max":
    # Lmax 선언
    L_max = model_MIP_1.addVar(vtype=GRB.CONTINUOUS, lb = 0, name= "L_max") # L_max를 나타내는 연속형 변수

    # Lmax 제약식
    constraint =  {(j): model_MIP_1.addConstr(L_max >= (C[j] - d[j]), name = f"constraint_{j}") for j in N}

    # 목적식
    model_MIP_1.setObjective(L_max, GRB.MINIMIZE)

#### To minimize the number of tardy jobs

$ Minimize \ \sum_{j=1}^n U_j$
$ \\ s.t. \ C_j \leq d_j + MU_j \quad \forall j \in N$
$ \\ U_j \in \{0, 1\} \quad \forall j \in N$

In [12]:
if obj_select == "tj":
    U =  {(j): model_MIP_1.addVar(vtype=GRB.BINARY, name=f"U_{j+1}") for j in N} # j가 Tardy job이면 1, 아니면 0인 이진변수

    # Tardy job 제약식
    constraint =  {(j): model_MIP_1.addConstr(C[j] <= d[j] + M*U[j], name = f"constraint_{j}") for j in N}

    # 목적식
    model_MIP_1.setObjective(quicksum(U[j] for j in N), GRB.MINIMIZE)

#### To minimize the total weighted tardiness

$ Minimize \ \sum_{j=1}^n w_jT_j$
$ \\ s.t. \ T_j \geq C_j - d_j \quad \forall j \in N$
$ \\ T_j \geq 0 \quad \forall j \in N$

In [13]:
if obj_select == "twt":
    T = {(j): model_MIP_1.addVar(vtype=GRB.CONTINUOUS, lb = 0, name=f"T_{j+1}") for j in N} # j에 대한 Tardiness 연속형 변수, lower bound 0 (0 이상)

    # Tardiness 제약식
    constraint =  {(j): model_MIP_1.addConstr(T[j] >= C[j] - d[j], name = f"constraint_{j}") for j in N}

    # 목적식
    model_MIP_1.setObjective(quicksum(w[j]*T[j] for j in N), GRB.MINIMIZE)

## 2. Time index variables
### Sets
- T: planning horizon ids discretized into the periods 1,2,3,...T

In [14]:
T = [_ for _ in range(sum(p) * 2)] # 넉넉하게 processing time 합 * 2 로 선택 -> processing time, release time이 늘어난다면 늘려줘야 함, T가 작을수록 공간복잡도 감소

### Decision Variables
- $x_{j,t}$: 작업 $j$가 period t에서 시작하면 1, 아니면 0인 이진변수, $x_{j,t} \in \{0, 1\} \ \forall j \in N, \forall t \in T$

In [15]:
model_MIP_2 = gp.Model("MIP_2") # 먼저 모델 선언

# colab 경우
# model_MIP_2 = gp.Model("MIP_2", env = env) 

# Decision Variable 만들기
x =  {(j, t): model_MIP_2.addVar(vtype=GRB.BINARY, name=f"x_{j+1}_{t}") for j in N for t in T} # 이진 변수, j < k 는 그대로 써주면 됨

### 기본 constraint
- $\sum_{t=1}^{|T|-p_j+1} x_{j,t} = 1 \quad \forall j \in N$
- $\sum_{j=1}^n \sum_{s = max(0, t-p_j+1)}^t x_{j,s} \quad \forall t \in T$

In [16]:
constraint1 = {(j): model_MIP_2.addConstr(quicksum(x[j,t] for t in range(0, len(T)-p[j]+1)) == 1, name = f"constraint1_{j}") for j in N}
constraint2 = {(t): model_MIP_2.addConstr(quicksum(quicksum(x[j,s] for s in range(max(0, t-p[j]+1), t + 1)) for j in N) <= 1, name = f"constraint2_{t}") for t in T}

#### Release time 반영 제약식
$ x_{j,t} = 0 \quad \forall t \leq r_j, \forall j \in J $

In [17]:
if release_time == True:
    constraint3 = {(j, t): model_MIP_2.addConstr(x[j,t] == 0, name = f"constraint3_{j}_{t}") for j in N for t in T if t + 1 <= r[j]}

#### To minimize the total weighted completion time
$ Minimize \sum_{j = 1}^n\sum_{t=1}^{|T|-p_j+1} \xi_{j,t} x_{j,t}$
$\\ \xi_{j,t} = w_j(t-1+p_j) \quad \forall j \in N, \forall t \in T$

In [18]:
if obj_select == "wct":
    # 크사이 데이터 셋을 리스트로 구현하여 사용
    xi = [[w[j]*(t + p[j]) for t in T] for j in N] # t = 0 부터 시작하기에 1을 빼줄 필요 없음 논문은 1부터 시작해서 그런 것임

    # 목적식
    model_MIP_2.setObjective(quicksum(quicksum(xi[j][t] * x[j,t] for t in range(0, len(T)-p[j]+1)) for j in N), GRB.MINIMIZE)

#### To minimize the number of tardy jobs
$ Minimize \sum_{j = 1}^n\sum_{t=1}^{|T|-p_j+1} \xi_{j,t} x_{j,t}$
$ \\ \xi_{j,t} = 1, \  if \  t > (d_j-p_j+1), \quad \forall j \in N, \forall t \in T \\ 0, \ otherwise $

In [19]:
if obj_select == "tj":
    # 크사이 데이터 셋을 리스트로 구현하여 사용
    xi = [[1 if t > (d[j] - p[j]) else 0 for t in T] for j in N] # t = 0 부터 시작하기에 1을 빼줄 필요 없음 논문은 1부터 시작해서 그런 것임

    # 목적식
    model_MIP_2.setObjective(quicksum(quicksum(xi[j][t] * x[j,t] for t in range(0, len(T)-p[j]+1)) for j in N), GRB.MINIMIZE)

#### To minimize the total weighted tardiness
$ Minimize \sum_{j = 1}^n\sum_{t=1}^{|T|-p_j+1} \xi_{j,t} x_{j,t}$
$ \\ \xi_{j,t} = w_j\max(0, t - 1 + p_j - d_j), \quad \forall j \in N, \forall t \in T$

In [20]:
if obj_select == "twt":
    # 크사이 데이터 셋을 리스트로 구현하여 사용
    xi = [[w[j]*max(0, t + p[j] - d[j]) for t in T] for j in N] # t = 0 부터 시작하기에 1을 빼줄 필요 없음 논문은 1부터 시작해서 그런 것임

    # 목적식
    model_MIP_2.setObjective(quicksum(quicksum(xi[j][t] * x[j,t] for t in range(0, len(T)-p[j]+1)) for j in N), GRB.MINIMIZE)

#### To minimize $L_{max}$
$ Minimize \ L_{max}$
$ \\ s.t. \  L_{max} \geq (C_j - d_j) \quad \forall j \in N$
$ \\ C_j = \sum_{t=1}^{|T|-p_j+1}(t-1+p_j)x_{j,t} \quad \forall j \in N$

In [21]:
if obj_select == "L_max":
    # Lmax 선언
    L_max = model_MIP_2.addVar(vtype=GRB.CONTINUOUS, lb = 0, name= "L_max") # L_max를 나타내는 연속형 변수

    # Lmax 제약식
    constraint =  {(j): model_MIP_2.addConstr(L_max >= (quicksum((t+p[j]) * x[j,t] for t in range(0, len(T)-p[j]+1))- d[j]), name = f"constraint_{j}") for j in N} # C_j 대신 바로 x제약식 넣어줌. 그리고 t = 0부터 시작하니까 t-1 해줄 필요 없음

    # 목적식
    model_MIP_2.setObjective(L_max, GRB.MINIMIZE)

## 3. Linear ordering variables

### Decision Variables
- $\delta_{j,k}$: 작업 $j$가 작업 $k$보다 앞에서 시작하면 1, 아니면 0인 이진변수, $\delta_{j,k} \in \{0, 1\} \ \forall j \in N, \forall k \in N$
- $S_j$: 작업 $j$의 시작 시점을 의미

In [22]:
model_MIP_3 = gp.Model("MIP_3")

# colab 경우
# model_MIP_3 = gp.Model("MIP_3", env = env) 

# Decision Variable 만들기
delta = {(j, k): model_MIP_3.addVar(vtype=GRB.BINARY, name=f"delta_{j+1}_{k+1}") for j in N for k in N}

if release_time == True:
    S = {(j): model_MIP_3.addVar(vtype = GRB.CONTINUOUS, lb = 0, name=f'S_{j+1}') for j in N}

### 기본 constraint
- $\delta_{j,k} + \delta_{k,j} = 1 \quad \forall j,k \in N, \ 1 \leq j < k \leq n,$, j<k 가 맞음. 등호 있을 시 infeasible
- $\delta_{j,k} + \delta_{k,l} + \delta_{l,j} \leq 2 \quad \forall j,k,l \in N, \ and \ j \neq k \neq l$

In [23]:
constraint1 = {(j, k): model_MIP_3.addConstr(delta[j,k] + delta[k,j] == 1, name = f"constraint1_{j}_{k}") for j in N for k in N if j < k}
constraint2 = {(j, k, l): model_MIP_3.addConstr(delta[j,k] + delta[k,l] + delta[l,j] <= 2, name = f"constraint2_{j}_{k}_{l}") for j in N for k in N for l in N if j != k and k != l and j != l}

#### Release time 반영 제약식
$ S_j \geq r_i \delta_{i,j} + \sum_{k < i, k\neq j}p_k(\delta_{i,k} + \delta_{k,j} -1) + \sum_{k \geq i, k \neq j}p_k \delta_{k,j} \quad \forall i, j \in N$
$ \\ \delta_{j,j} = 1, \quad \forall j \in N$
#### 논문의 식 제대로 작동하지 않았음. 새로운 아래 수식으로 대체
$ \\ S_j \geq r_j \quad \forall j \in N$
$ \\ S_k \geq S_j + p_j - M(1-\delta_{j,k}) \quad \forall j \in N, \forall k \in N, j \neq k$
$ \\ S_j \geq S_k + p_k - M\delta_{j,k}  \quad \forall j \in N, \forall k \in N, j \neq k$

In [24]:
if release_time == True:
    # constraint3 = {(i,j): model_MIP_3.addConstr(S[j] >= r[i]*delta[i,j] + quicksum(p[k] * (delta[i,k] + delta[k,j] - 1) for k in range(0, i) if k != j) + quicksum(p[k]*delta[k,j] for k in range(i, n) if k != j), name = f"constraint3_{i}_{j}") for i in N for j in N}
    # constraint4 = {(j): model_MIP_3.addConstr(delta[j,j] == 1, name = f"constraint4_{j}") for j in N}
    constraint3 = {(j): model_MIP_3.addConstr(S[j] >= r[j], name = f"constraint_{j}") for j in N}
    constraint4 = {(j, k): model_MIP_3.addConstr(S[k] >= S[j] + p[j] - M*(1-delta[j,k]), name = f"constraint4_{j}_{k}") for j in N for k in N if j != k}
    constraint4 = {(j, k): model_MIP_3.addConstr(S[j] >= S[k] + p[k] - M*(delta[j,k]), name = f"constraint5_{j}_{k}") for j in N for k in N if j != k}

#### To minimize the total weighted completion time
$ Minimize \sum_{j,k \in N, k \neq j}w_jp_j\delta_{k,j} + \sum_{j \in N}w_jp_j$
#### To minimize the total weighted completion time with release time
$ Minimize \sum_{j \in N}w_j(S_j + p_j)$

In [25]:
if obj_select == 'wct':
    if release_time == False:
        # 목적식
        model_MIP_3.setObjective(quicksum(w[j]*p[k]*delta[k,j] for j in N for k in N if k != j) + quicksum(w[j]*p[j] for j in N), GRB.MINIMIZE)
    elif release_time == True:
        model_MIP_3.setObjective(quicksum(w[j]* (S[j] + p[j]) for j in N), GRB.MINIMIZE)

#### To minimize $L_{max}$
$ Minimize \ L_{max}$
$ \\ s.t. \  L_{max} \geq (C_j - d_j) \quad \forall j \in N$
$ \\ C_j = \sum_{k \in N, k \neq j}p_k\delta_{k,j} + p_j \quad \forall j \in N$
#### To minimize $L_{max}$ with release time
$ \\ s.t. \  L_{max} \geq S_j + p_j - d_j \quad \forall j \in N$

In [26]:
if obj_select == "L_max":
    # Lmax 선언
    L_max = model_MIP_3.addVar(vtype=GRB.CONTINUOUS, lb = 0, name= "L_max") # L_max를 나타내는 연속형 변수

    if release_time == False:
        # Lmax 제약식
        constraint = {(j): model_MIP_3.addConstr(L_max >= (quicksum(p[k]*delta[k,j] for k in N if k != j) + p[j] - d[j]), name = f"constraint_{j}") for j in N} # C_j 대신 바로 x제약식 넣어줌. 그리고 t = 0부터 시작하니까 t-1 해줄 필요 없음
    elif release_time == True:
        constraint = {(j): model_MIP_3.addConstr(L_max >= S[j] + p[j] - d[j], name = f"constraint_{j}") for j in N}

    # 목적식
    model_MIP_3.setObjective(L_max, GRB.MINIMIZE)

#### To minimize the number of tardy jobs

$ Minimize \ \sum_{j=1}^n U_j$
$ \\ s.t. \ C_j \leq d_j + MU_j \quad \forall j \in N$
$ \\ U_j \in \{0, 1\} \quad \forall j \in N$
$ \\ C_j = \sum_{k \in N, k \neq j}p_k\delta_{k,j} + p_j \quad \forall j \in N$

#### To minimize the number of tardy jobs release time
$ \\ s.t. \  S_j + p_j \leq d_j + MU_j \quad \forall j \in N $

In [27]:
if obj_select == "tj":
    U =  {(j): model_MIP_3.addVar(vtype=GRB.BINARY, name=f"U_{j+1}") for j in N} # j가 Tardy job이면 1, 아니면 0인 이진변수

    if release_time == False:
        # Tardy job 제약식
        constraint = {(j): model_MIP_3.addConstr(quicksum(p[k]*delta[k,j] for k in N if k != j) + p[j] <= d[j] + M*U[j], name = f"constraint_{j}") for j in N}
    elif release_time == True:
        constraint = {(j): model_MIP_3.addConstr(S[j] + p[j] <= d[j] + M*U[j], name = f"constraint_{j}") for j in N}

    # 목적식
    model_MIP_3.setObjective(quicksum(U[j] for j in N), GRB.MINIMIZE)

#### To minimize the total weighted tardiness

$ Minimize \ \sum_{j=1}^n w_jT_j$
$ \\ s.t. \ T_j \geq C_j - d_j \quad \forall j \in N$
$ \\ T_j \geq 0 \quad \forall j \in N$
$ \\ C_j = \sum_{k \in N, k \neq j}p_k\delta_{k,j} + p_j \quad \forall j \in N$

#### To minimize the total weighted tardiness with release time

$ \\ s.t. \ T_j \geq S_j + p_j -d_j \quad \forall j \in N$

In [28]:
if obj_select == "twt":
    T = {(j): model_MIP_3.addVar(vtype=GRB.CONTINUOUS, lb = 0, name=f"T_{j+1}") for j in N} # j에 대한 Tardiness 연속형 변수, lower bound 0 (0 이상)

    if release_time == False:
        # Tardiness 제약식
        constraint = {(j): model_MIP_3.addConstr(T[j] >= quicksum(p[k]*delta[k,j] for k in N if k != j) + p[j] - d[j], name = f"constraint_{j}") for j in N}
    elif release_time == True:
        constraint = {(j): model_MIP_3.addConstr(T[j] >= S[j] + p[j] - d[j], name = f"constraint_{j}") for j in N}

    # 목적식
    model_MIP_3.setObjective(quicksum(w[j]*T[j] for j in N), GRB.MINIMIZE)

## 4. Assignment and positional date variables

### Decision Variables
- $u_{j,k}$: 작업 $j$가 position k에 있으면 1, 아니면 0인 이진변수, $u_{j,k} \in \{0, 1\} \ \forall j \in N, \forall k \in N$
- $\gamma_j$: 작업 $j$의 completion time

In [29]:
model_MIP_4 = gp.Model("MIP_4")

# colab 경우
# model_MIP_4 = gp.Model("MIP_4", env = env) 

# Decision Variable 만들기
u = {(j,k): model_MIP_4.addVar(vtype = GRB.BINARY, name = f"u_{j}_{k}") for j in N for k in N}
gamma = {(j): model_MIP_4.addVar(vtype=GRB.CONTINUOUS, lb = 0, name=f"gamma_{j+1}") for j in N}

### 기본 constraint
- $\sum_{k \in N}u_{j,k} = 1 \quad \forall j \in N$
- $\sum_{j \in N}u_{j,k} = 1 \quad \forall k \in N$
- $\gamma_j \geq \sum_{j \in N}p_ju_{j,1} $
- $\gamma_k \geq \gamma_{k-1} + \sum_{j \in N}p_ju_{j,k} \quad \forall k = 2,3,..N$


In [30]:
constraint1 = {(j): model_MIP_4.addConstr(quicksum(u[j,k] for k in N) == 1, name = f"constraint1_{j}") for j in N}
constraint2 = {(k): model_MIP_4.addConstr(quicksum(u[j,k] for j in N) == 1, name = f"constraint2_{k}") for k in N}
constraint3 = model_MIP_4.addConstr(gamma[0] >= quicksum(p[j]*u[j,0] for j in N))
constraint4 = {(k): model_MIP_4.addConstr(gamma[k] >= gamma[k-1] + quicksum(p[j]*u[j,k] for j in N), name = f"constraint4_{k}") for k in N[1:]}

#### Release time 반영 제약식
$\gamma_k \geq \sum_{j \in N}(p_j + r_j)u_{j,k} \quad \forall k \in N$

In [31]:
if release_time == True:
    constraint5 = {(k): model_MIP_4.addConstr(gamma[k] >= quicksum((p[j] + r[j]) * u[j,k] for j in N), name = f"constraint5_{k}") for k in N}

#### To minimize the number of tardy jobs

$ Minimize \ \sum_{k=1}^n U_k$
$ \\ \gamma_k \leq \sum_{j \in N}d_ju_{j,k} + MU_k \quad \forall k \in N$

In [32]:
if obj_select == "tj":
    U =  {(j): model_MIP_4.addVar(vtype=GRB.BINARY, name=f"U_{j+1}") for j in N} # j가 Tardy job이면 1, 아니면 0인 이진변수

    # Tardy job 제약식
    constraint =  {(k): model_MIP_4.addConstr(gamma[k] <= quicksum(d[j]*u[j,k] for j in N) + M*U[k], name = f"constraint_{k}") for k in N}

    # 목적식
    model_MIP_4.setObjective(quicksum(U[j] for j in N), GRB.MINIMIZE)

#### To minimize $L_{max}$
$ Minimize \ L_{max}$
$ \\ s.t. \  L_{max} \geq \gamma_k - \sum_{j \in N} d_ju_{j,k} \quad \forall k \in N$

In [33]:
if obj_select == "L_max":
    # Lmax 선언
    L_max = model_MIP_4.addVar(vtype=GRB.CONTINUOUS, lb = 0, name= "L_max") # L_max를 나타내는 연속형 변수

    # Lmax 제약식
    constraint = {(k): model_MIP_4.addConstr(L_max >= gamma[k] - quicksum(d[j]*u[j,k] for j in N), name = f"constraint_{k}") for k in N} # C_j 대신 바로 x제약식 넣어줌. 그리고 t = 0부터 시작하니까 t-1 해줄 필요 없음

    # 목적식
    model_MIP_4.setObjective(L_max, GRB.MINIMIZE)

#### To minimize the total weighted completion time
$ Minimize \sum_{j = 1}^n w_j C_j$
$\\ C_j \geq \gamma_k - M(1-u_{j,k}) \quad \forall j,k \in N$
$\\ C_j \geq 0 \quad \forall j \in N$

In [34]:
if obj_select == "wct":
    # Lmax 선언
    C = {(j): model_MIP_4.addVar(vtype=GRB.CONTINUOUS, lb = 0, name=f"C_{j+1}") for j in N}

    # Lmax 제약식
    constraint = {(j, k): model_MIP_4.addConstr(C[j] >= gamma[k] - M*(1-u[j,k]), name = f"constraint_{j}_{k}") for j in N for k in N} # C_j 대신 바로 x제약식 넣어줌. 그리고 t = 0부터 시작하니까 t-1 해줄 필요 없음

    # 목적식
    model_MIP_4.setObjective(quicksum(w[j]*C[j] for j in N), GRB.MINIMIZE)

#### To minimize the total weighted tardiness

$ Minimize \ \sum_{j=1}^n w_jT_j$
$\\ C_j \geq \gamma_k - M(1-u_{j,k}) \quad \forall j,k \in N$
$\\ C_j \geq 0 \quad \forall j \in N$
$\\ T_j \geq C_j - d_j \quad \forall j \in N$
$\\ T_j \geq 0 $

In [35]:
if obj_select == "twt":
    # Lmax 선언
    C = {(j): model_MIP_4.addVar(vtype=GRB.CONTINUOUS, lb = 0, name=f"C_{j+1}") for j in N}
    T = {(j): model_MIP_4.addVar(vtype=GRB.CONTINUOUS, lb = 0, name=f"T_{j+1}") for j in N}

    # Lmax 제약식
    constraint = {(j, k): model_MIP_4.addConstr(C[j] >= gamma[k] - M*(1-u[j,k]), name = f"constraint_{j}_{k}") for j in N for k in N} # C_j 대신 바로 x제약식 넣어줌. 그리고 t = 0부터 시작하니까 t-1 해줄 필요 없음
    constraint = {(j): model_MIP_4.addConstr(T[j] >= C[j]-d[j], name = f"constraint_{j}") for j in N}

    # 목적식
    model_MIP_4.setObjective(quicksum(w[j]*T[j] for j in N), GRB.MINIMIZE)

In [36]:
model_MIP_1.optimize()
model_MIP_2.optimize()
model_MIP_3.optimize()
model_MIP_4.optimize()

Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 13th Gen Intel(R) Core(TM) i9-13900K, instruction set [SSE2|AVX|AVX2]
Thread count: 24 physical cores, 32 logical processors, using up to 32 threads

Optimize a model with 240 rows, 135 columns and 675 nonzeros
Model fingerprint: 0x88bec88b
Variable types: 30 continuous, 105 integer (105 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+03]
  Objective range  [1e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [9e+00, 7e+03]
Presolve removed 15 rows and 0 columns
Presolve time: 0.00s
Presolved: 225 rows, 135 columns, 660 nonzeros
Variable types: 30 continuous, 105 integer (105 binary)
Found heuristic solution: objective 6636.0000000

Root relaxation: objective 0.000000e+00, 105 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Ti

In [37]:
# 목적식 value
print(model_MIP_1.ObjVal)
print(model_MIP_2.ObjVal)
print(model_MIP_3.ObjVal)
print(model_MIP_4.ObjVal)

2504.0
2504.0
2504.0
2504.0


In [38]:
# 문제 푸는 데 걸린 시간
print(round(model_MIP_1.RunTime, 3))
print(round(model_MIP_2.RunTime, 3))
print(round(model_MIP_3.RunTime, 3))
print(round(model_MIP_4.RunTime, 3))

0.292
2.048
0.089
1.615


In [39]:
# Objective solution 출력 과정
for v in model_MIP_1.getVars():
    if v.x > 1e-6:
        print(v.varName, v.x)

print("Objective value:", model_MIP_1.objVal)

C_1 696.0
C_2 220.0
C_3 586.0
C_4 608.0
C_5 492.0
C_6 521.0
C_7 347.0
C_8 338.0
C_9 12.0
C_10 508.0
C_11 238.0
C_12 129.0
C_13 429.0
C_14 67.0
C_15 268.0
y_2_3 1.0
y_2_4 1.0
y_2_5 1.0
y_2_6 1.0
y_2_7 1.0
y_2_8 1.0
y_2_10 1.0
y_2_11 1.0
y_2_13 1.0
y_2_15 1.0
y_3_4 1.0
y_5_6 1.0
y_5_10 1.0
y_7_10 1.0
y_7_13 1.0
y_8_10 1.0
y_8_13 1.0
y_9_10 1.0
y_9_11 1.0
y_9_12 1.0
y_9_13 1.0
y_9_14 1.0
y_9_15 1.0
y_11_13 1.0
y_11_15 1.0
y_12_13 1.0
y_12_15 1.0
y_14_15 1.0
T_1 120.0
T_2 34.0
T_3 59.0
T_5 413.0
T_8 57.0
T_10 11.0
T_12 26.0
T_13 38.0
Objective value: 2504.0


In [40]:
# Objective solution 출력 과정
for v in model_MIP_2.getVars():
    if v.x > 1e-6:
        print(v.varName, v.x)

print("Objective value:", model_MIP_2.objVal)

x_1_608 1.0
x_2_129 1.0
x_3_521 1.0
x_4_586 1.0
x_5_429 1.0
x_6_508 1.0
x_7_338 1.0
x_8_268 1.0
x_9_0 1.0
x_10_492 1.0
x_11_220 1.0
x_12_67 1.0
x_13_347 1.0
x_14_12 1.0
x_15_238 1.0
Objective value: 2504.0


In [41]:
# Objective solution 출력 과정
for v in model_MIP_3.getVars():
    if v.x > 1e-6:
        print(v.varName, v.x)

print("Objective value:", model_MIP_3.objVal)

delta_2_1 1.0
delta_2_3 1.0
delta_2_4 1.0
delta_2_5 1.0
delta_2_6 1.0
delta_2_7 1.0
delta_2_8 1.0
delta_2_10 1.0
delta_2_11 1.0
delta_2_13 1.0
delta_2_15 1.0
delta_3_1 1.0
delta_3_4 1.0
delta_4_1 1.0
delta_5_1 1.0
delta_5_3 1.0
delta_5_4 1.0
delta_5_6 1.0
delta_5_10 1.0
delta_6_1 1.0
delta_6_3 1.0
delta_6_4 1.0
delta_7_1 1.0
delta_7_3 1.0
delta_7_4 1.0
delta_7_5 1.0
delta_7_6 1.0
delta_7_10 1.0
delta_7_13 1.0
delta_8_1 1.0
delta_8_3 1.0
delta_8_4 1.0
delta_8_5 1.0
delta_8_6 1.0
delta_8_7 1.0
delta_8_10 1.0
delta_8_13 1.0
delta_9_1 1.0
delta_9_2 1.0
delta_9_3 1.0
delta_9_4 1.0
delta_9_5 1.0
delta_9_6 1.0
delta_9_7 1.0
delta_9_8 1.0
delta_9_10 1.0
delta_9_11 1.0
delta_9_12 1.0
delta_9_13 1.0
delta_9_15 1.0
delta_10_1 1.0
delta_10_3 1.0
delta_10_4 1.0
delta_10_6 1.0
delta_11_1 1.0
delta_11_3 1.0
delta_11_4 1.0
delta_11_5 1.0
delta_11_6 1.0
delta_11_7 1.0
delta_11_8 1.0
delta_11_10 1.0
delta_11_13 1.0
delta_11_15 1.0
delta_12_1 1.0
delta_12_2 1.0
delta_12_3 1.0
delta_12_4 1.0
delta_12_5 1.

In [42]:
# Objective solution 출력 과정
for v in model_MIP_4.getVars():
    if v.x > 1e-6:
        print(v.varName, v.x)

print("Objective value:", model_MIP_4.objVal)

u_0_14 1.0
u_1_3 1.0
u_2_12 1.0
u_3_13 1.0
u_4_9 1.0
u_5_11 1.0
u_6_7 1.0
u_7_6 1.0
u_8_0 1.0
u_9_10 1.0
u_10_4 1.0
u_11_2 1.0
u_12_8 1.0
u_13_1 1.0
u_14_5 1.0
gamma_1 12.0
gamma_2 67.0
gamma_3 129.0
gamma_4 220.0
gamma_5 238.0
gamma_6 268.0
gamma_7 338.0
gamma_8 347.0
gamma_9 429.0
gamma_10 492.0
gamma_11 508.0
gamma_12 521.0
gamma_13 586.0
gamma_14 608.0
gamma_15 696.0
C_1 696.0
C_2 220.0
C_3 586.0
C_4 608.0
C_5 492.0
C_6 521.0
C_7 347.0
C_8 338.0
C_9 12.0
C_10 508.0
C_11 238.0
C_12 129.0
C_13 429.0
C_14 67.0
C_15 268.0
T_1 120.0
T_2 34.0
T_3 59.0
T_5 413.0
T_8 57.0
T_10 11.0
T_12 26.0
T_13 38.0
Objective value: 2504.0
